#Бенчмарк. Определение остановок автобусов.

Ночь Анализа Данных Яндекса, 2015

Подключим необходимые библиотеки:

In [1]:
import numpy as np
import pandas as pd

from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier

Считаем файлы с обучающей и тестовой выборкой, а также файл с остановками.

In [2]:
dtrain = pd.read_csv(
    'gps_train.tsv',
    sep='\t',
    names=['datetime', 'id', 'type', 'hash', 'latitude', 'longitude'])

dtest = pd.read_csv(
    'gps_test.tsv',
    sep='\t',
    names=['datetime', 'id', 'type', 'hash', 'latitude', 'longitude'])

dstops = pd.read_csv(
    'stops_train.tsv', 
    sep='\t',
    names=['latitude', 'longitude'])

Создадим массивы с координатами точек.

In [3]:
coordtrain = np.array(dtrain.ix[:, 4:])
coordtest = np.array(dtest.ix[:, 4:])

coordstops = np.array(dstops.ix[:, :])

Создадим обучающую выборку.
Для этого пометим как остановкии те точки маршрутов, которые наиболее близки к реальным остановкам.

In [4]:
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(coordtrain)
selected = nbrs.kneighbors(coordstops)[1][:, 0]

ytrain = np.zeros(len(dtrain))
ytrain[selected] = 1

В качестве признаков возьмем расстояния до 5 ближайших точек из обучающей и тестовой выборки соответственно.

In [5]:
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(coordtrain)
Xtrain = nbrs.kneighbors(coordtrain)[0]

nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(coordtest)
Xtest = nbrs.kneighbors(coordtest)[0]

Обучим модель случайного леса на наших данных.

In [6]:
model = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1)
model.fit(Xtrain, ytrain)

result = model.predict_proba(Xtest)[:, 1]

В качестве результата возьмем 3000 наиболее вероятных точек.
Выведем результат в файл.

In [8]:
np.savetxt('output.txt', coordtest[np.argsort(result)[-3000:]], header='3000', comments='')

In [9]:
!head output.txt

3000
5.567736500000000177e+01 3.722490100000000268e+01
5.560834899999999692e+01 3.761362700000000103e+01
5.557557299999999856e+01 3.774151499999999970e+01
5.561267800000000250e+01 3.765497199999999367e+01
5.549281700000000228e+01 3.758671799999999763e+01
5.550112900000000593e+01 3.748827199999999493e+01
5.550378100000000359e+01 3.748466400000000220e+01
5.554626999999999271e+01 3.761847199999999702e+01
5.549106400000000150e+01 3.753749600000000441e+01


In [10]:
oauth='21a1929c450641769be5c5bc49a55d54'

In [11]:
from contest_yandex import YaContestSubmitter
contest = YaContestSubmitter(oauth_token=oauth, contest_id=1304)

In [12]:
r = contest.submit('output.txt')
print r

770171


In [13]:
score, message = contest.get_score()
print "Score: {}, message: {}".format(score, message)

Score: 97740.2536205, message: points 97740.25362053150 [*] Correct stops 3 of 1143 in public set.

